In [21]:
import gradio as gr
from transformers import pipeline,BartForConditionalGeneration, AutoTokenizer

In [11]:
loaded_model = BartForConditionalGeneration.from_pretrained("ManuVleuBeu/BART_answer-aware_MSMARCO").to("cuda:0")
tokenizer = AutoTokenizer.from_pretrained("ManuVleuBeu/BART_answer-aware_MSMARCO")

In [22]:
def run_model(input_string,target_answer, **generator_args):
    generator_args = {
    "max_length": 768,
    "num_beams": 4,# bij grotere num_beams is trager maar complexere vragen(niet per se betere vragen)
    "length_penalty": 1.5,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    }
    
    input_string = input_string + " " + "<ANSWER>" + target_answer + " </s>"
    input_ids = tokenizer.encode(input_string, return_tensors="pt")
    res = loaded_model.generate(input_ids.to("cuda:0"), **generator_args)
    output = tokenizer.batch_decode(res, skip_special_tokens=True)
    output = [item.split("<sep>") for item in output]
    return output

In [23]:
import os
import markdown
from bs4 import BeautifulSoup
import pickle
from utils import Chapter, File


In [ ]:
GF_FOLDER = os.path.join("C:", os.sep, "Users", "manuv", "OneDrive", "Documenten", "Code", "BAP", "BP_gitrepo", "BP_Info_Support", "docs")
STRUCTURE_FILE = "structure.md"
CHAPTER_TREE = Chapter("root",GF_FOLDER)

SAVE_TREE_DIR = os.path.join("C:", os.sep, "Users", "manuv", "OneDrive", "Documenten", "Code", "BAP", "BP_gitrepo", "BP_Info_Support", "data")

In [24]:
GF_FOLDER = os.path.join("E:", os.sep, "HOGENT", "2022_2023", "BA","docs")
STRUCTURE_FILE = "structure.md"
CHAPTER_TREE = Chapter("root",GF_FOLDER)
SAVE_TREE_DIR = os.path.join("E:", os.sep, "HOGENT", "2022_2023", "BA","docs")


In [25]:
def get_guidance_framework():
    with open(os.path.join(SAVE_TREE_DIR,"gf_structure.pkl"), "rb") as f:
        root = pickle.load(f)

In [26]:
def find_chapter_by_dir(root, dir):
    if root.dir == dir:
        return root
    for child in root.children:
        found_chapter = find_chapter_by_dir(child, dir)
        if found_chapter:
            return found_chapter
    return None

In [27]:
rnd_chapter = CHAPTER_TREE.get_random_chapter()
print(f'Chapter Name: {rnd_chapter.name}')
rnd_file = rnd_chapter.get_random_file()
print(f'File Name: {rnd_file.name}')
print(f'File Dir: {rnd_file.dir}')
print('Structured Text in file')
rnd_file.text

Chapter Name: root


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 rnd_chapter = CHAPTER_TREE.get_random_chapter()                                              │
│   2 print(f'Chapter Name: {rnd_chapter.name}')                                                   │
│   3 rnd_file = rnd_chapter.get_random_file()                                                     │
│ ❱ 4 print(f'File Name: {rnd_file.name}')                                                         │
│   5 print(f'File Dir: {rnd_file.dir}')                                                           │
│   6 print('Structured Text in file')                                                             │
│   7 rnd_file.text                                                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NoneType' object has no attribute 'name'

### User Interface with Gradio for QG & QA

In [20]:
import gradio as gr
from transformers import pipeline
import Levenshtein

# Question Generation Model
question_generator = pipeline("text2text-generation", model="ManuVleuBeu/T5_ag_SQuAD")
# question_generator = pipeline("text2text-generation", model="valhalla/t5-base-qa-qg-hl")

# Answer Generation Model
answer_generator = pipeline("question-answering", model="damapika/roberta-base_mod_squad")
# TODO: Add all chapter contexts or an extra function where user selects chapter->context
contexts = [
    "In the world of literature, there have been many authors who have gained a reputation for their ability to create complex characters. One such author is Fyodor Dostoevsky, a Russian novelist who wrote several influential works in the 19th century.", 
    "My name is Sarah and I live in London."
]  # Add your contexts to this list
selected_context=[None]
generated_question=[None]
generated_answer=[None]

def calculate_similarity(answer1, answer2):
    similarity_score = 1 - (Levenshtein.distance(answer1.lower(), answer2.lower()) / max(len(answer1), len(answer2)))
    return similarity_score

def generate_question(context):
    selected_context[0] = context
    question = question_generator(context)[0]['generated_text']
    question = question.split('?')[0]+'?'
    generated_question[0]=question
    return question

def generate_answer(user_answer):
    question = generated_question[0]
    context = selected_context[0]
    answer = answer_generator(context=context,question=question)
    print(answer['answer'])
    generated_answer[0] = answer['answer']
    score=calculate_similarity(user_answer,generated_answer[0])
    return [user_answer,answer['answer'],score]

# Interface for QG
io1 = gr.Interface(
    fn=generate_question,
    inputs=[
        gr.inputs.Dropdown(choices=contexts, label="Select a context", type="value"),
    ],
    live=False,
    outputs=[
        gr.outputs.Textbox(label="Generated Question")
    ],
    title="Quiz Generator",
    description="Select a context, generate a question, and compare your answer.",
    allow_flagging="never"

)

# Additional interface for answer checking
io2 = gr.Interface(
    fn=generate_answer,
    live=False,
    inputs=[gr.outputs.Textbox(label="User Answer")],
    outputs=[gr.outputs.Textbox(label="User Answer"),gr.outputs.Textbox(label="Generated Answer"),gr.outputs.Textbox(label="Levenshtein Answer score")],
    title="Answer Checker",
    description="Compare your answer.",
        allow_flagging="never"

)

io1.launch()
io2.launch()



c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:219: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\inputs.py:222: UserWarning: `optional` parameter is deprecated, and it has no effect
  super().__init__(
c:\Users\dama_\AppData\Local\Programs\Python\Python311\Lib\site-packages\gradio\outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Running on local URL:  http://127.0.0.1:7885

To create a public link, set `share=True` in `launch()`.


Running on local URL:  http://127.0.0.1:7886

To create a public link, set `share=True` in `launch()`.


Fyodor Dostoevsky
